In [265]:
import pandas as pd
import numpy as np

In [266]:
df = pd.read_csv("/PATH/OF/OUTPUT/final_outs.csv")
df_true = pd.read_csv("/PATH/OF/TEST/SET/df_test.csv")

In [270]:
classes = [
    'any cancer',
    'progression',
    'response',
    'brain',
    'bone',
    'adrenal',
    'liver',
    'lung',
    'lymph node',
    'peritoneal'
]

col_names = [
        ['any_cancer', 'any cancer'],
        ['progression', 'progression'],
        ['response', 'response'],
        ['brain_met', 'brain'],
        ['bone_met', 'bone'],
        ['adrenal_met', 'adrenal'],
        ['liver_met', 'liver/hepatic'],
        ['lung_met', 'lung'],
        ['node_met', 'lymph'],
        ['peritoneal_met', 'peritoneal']
    ]

In [272]:
gen_responses = []

import re

for i in range(len(df)):
    exc_dict = {}
    response = df["output"][i].split("\n")[:11]
    for c in classes:
        for res in response:
            if c in res.lower():
                if int(re.findall(r'\d+', res)[-1]) > 1:
                    exc_dict[c] = None
                else:
                    exc_dict[c] = int(re.findall(r'\d+', res)[-1])
        if c not in exc_dict.keys():
            exc_dict[c] = None
    
    assert len(exc_dict) == 10
    gen_responses.append(exc_dict)

print(gen_responses[4])


{'any cancer': 1, 'progression': 0, 'response': 1, 'brain': 0, 'bone': 1, 'adrenal': 0, 'liver': 0, 'lung': 0, 'lymph node': 0, 'peritoneal': 0}


In [273]:
any_cancer = [ele["any cancer"] for ele in gen_responses]
progression = [ele["progression"] for ele in gen_responses]
response = [ele["response"] for ele in gen_responses]
brain = [ele["brain"] for ele in gen_responses]
bone = [ele["bone"] for ele in gen_responses]
adrenal = [ele["adrenal"] for ele in gen_responses]
liver = [ele["liver"] for ele in gen_responses]
lung = [ele["lung"] for ele in gen_responses]
lymph_node = [ele["lymph node"] for ele in gen_responses]
peritoneal = [ele["peritoneal"] for ele in gen_responses]


results = [[a1,a2,a3,a4,a5,a6,a7,a8,a9,a10] for (a1,a2,a3,a4,a5,a6,a7,a8,a9,a10) in zip(any_cancer, progression, response, brain, bone, adrenal, liver, lung, lymph_node, peritoneal)]
df_pred = pd.DataFrame(results, columns =[ 'any_cancer', 'progression',
       'response', 'brain_met', 'bone_met', 'adrenal_met', 'liver_met',
       'lung_met', 'node_met', 'peritoneal_met'])


In [ ]:
null_mask = df_pred.isnull().any(axis=1)
null_rows = df_pred[null_mask]
null_rows

In [157]:
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import json
import os

def report(df_true, df_pred, dir, col_names:list):
    if not os.path.exists(dir):
        os.makedirs(dir)
    for cls in col_names:
        res = classification_report(df_true[cls[0]], df_pred[cls[0]], digits=3, output_dict=True)
        with open(f"{dir}/{cls[0]}.json", "w") as fp:
            json.dump(res , fp)



In [ ]:
dir = "/PATH/OF/RESULTS/NAME"
                
col_names = [
        ['any_cancer', 'any cancer_predicted'],
        ['progression', 'progression/worsening_predicted'],
        ['response', 'response/improvement_predicted'],
        ['brain_met', 'brain metastases_predicted'],
        ['bone_met', 'bone/osseous metastases_predicted'],
        ['adrenal_met', 'adrenal metastases_predicted'],
        ['liver_met', 'liver/hepatic metastases_predicted'],
        ['lung_met', 'lung/pulmonary metastases_predicted'],
        ['node_met', 'lymph node/nodal metastases_predicted'],
        ['peritoneal_met', 'peritoneal metastases_predicted']
    ]

report(df_true, df_pred, dir, col_names)